In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle

from Network import Network
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from GridSearch import GridSearcher
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from activation_functions import sigmoid, relu,relu_derivative, sigmoid_derivative, tanh, tanh_derivative
from sklearn.neural_network import MLPClassifier

In [ ]:
data=pd.read_csv('spam_final_df.data')
data.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=26)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
learning_rate=0.1

In [ ]:
# network = Network(learning_rate, epochs=1000, node_counts = [3, 3, 3])

In [ ]:
# params = {
#     "learning_rate": [0.1, 0.2, 0.5],
#     "epochs": [10, 100],
#     "node_counts": [[1], [2], [3], [3, 3], [3, 3, 3]],
# }
# gs = GridSearcher(network, params, X_train, y_train)
# gs_results = gs.search()
# gs_results

In [ ]:
network = Network(learning_rate=0.1)

In [ ]:
parameters = {
    'learning_rate': [0.1, 0.5],
    'epochs': [1000,2000],
    'node_counts': [[3,3],[10,10],[100,100],[3,3,3],[10,10,10],[20,20,20]],
    'activation_function': [(sigmoid,sigmoid_derivative),(relu,relu_derivative),(tanh,tanh_derivative)],

}
gs = GridSearcher(network, parameters, X_train, y_train)
gs_results = gs.search()
gs_results

In [ ]:
self_max = gs_results["score_mean"].idxmax()
print(gs_results.loc[self_max]["params"])
print(gs_results.loc[self_max]["score_mean"])

In [ ]:
network.set_params(gs_results.loc[self_max]["params"])
network.fit(X_train, y_train)
test_pred = network.predict(X_test)

In [ ]:
print(classification_report(y_test, test_pred))

In [ ]:
parameters_sk = {
    'learning_rate_init': [0.01, 0.1,0.5],
    "learning_rate": ["constant"],
    'max_iter': [1000,2000],
    'hidden_layer_sizes': [[3,3],[10,10],[100,100],[3,3,3],[10,10,10],[20,20,20]],
    'activation': ["logistic", "relu", "tanh"],
}

In [ ]:
sk_nn = MLPClassifier()

gs_sk = GridSearcher(sk_nn, parameters_sk, X_train, y_train)

In [ ]:
gs_results_sk = gs_sk.search()

In [ ]:
sk_max = gs_results_sk["score_mean"].idxmax()
print(gs_results_sk.loc[sk_max]["params"])
print(gs_results_sk.loc[sk_max]["score_mean"])

In [ ]:
sk_nn.set_params(**gs_results_sk.loc[sk_max]["params"])
sk_nn.fit(X_train, y_train)
test_pred_sk = sk_nn.predict(X_test)

In [ ]:
print(classification_report(y_test, test_pred_sk))